In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
import math
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import preprocess_input
import matplotlib.pyplot as plt

In [ ]:
data, info = tfds.load("imagenette/160px-v2", with_info=True, as_supervised=True)
train_data, valid_data = data['train'], data['validation']

del data

In [ ]:
train_dataset = train_data.map(
    lambda image, label: (tf.image.resize(image, (160, 160)), label))

validation_dataset = valid_data.map(
    lambda image, label: (tf.image.resize(image, (160, 160)), label)
)

del train_data
del valid_data

In [ ]:
num_classes = info.features['label'].num_classes
print(f'Total number of classes in dataset is {num_classes}')

Total number of classes in dataset is 10


In [ ]:
get_label_name = info.features['label'].int2str
text_labels = [get_label_name(i) for i in range(num_classes)]
for idx,i in enumerate(text_labels):
    print(f'The Label {idx} name is `{i}`')

The Label 0 name is `n01440764`
The Label 1 name is `n02102040`
The Label 2 name is `n02979186`
The Label 3 name is `n03000684`
The Label 4 name is `n03028079`
The Label 5 name is `n03394916`
The Label 6 name is `n03417042`
The Label 7 name is `n03425413`
The Label 8 name is `n03445777`
The Label 9 name is `n03888257`


In [ ]:
%%time

X_train = list(map(lambda x: x[0], train_dataset))
y_train = list(map(lambda x: x[1], train_dataset))


X_valid = list(map(lambda x: x[0], validation_dataset))
y_valid = list(map(lambda x: x[1], validation_dataset))

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_valid = tf.keras.utils.to_categorical(y_valid, num_classes)

del train_dataset
del validation_dataset

CPU times: user 23.9 s, sys: 5.18 s, total: 29 s
Wall time: 31.6 s


In [ ]:
train_len = info.splits['train'].num_examples
valid_len = info.splits['validation'].num_examples
print(f'Train size {train_len} and Valid size {valid_len}')

Train size 9469 and Valid size 3925


In [ ]:
y_train.shape,y_valid.shape

((9469, 10), (3925, 10))

In [ ]:
%%time
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      height_shift_range=0.2)

valid_datagen = ImageDataGenerator(
      rescale=1./255)


train_ds = tf.keras.preprocessing.image.NumpyArrayIterator(
    x=preprocess_input(np.array(X_train)), y=np.array(y_train), image_data_generator=train_datagen,batch_size=16
)

valid_ds = tf.keras.preprocessing.image.NumpyArrayIterator(
    x=preprocess_input(np.array(X_valid)), y=np.array(y_valid), image_data_generator=valid_datagen,batch_size=32
)

# train_datagen.fit(X_train)

CPU times: user 2.34 s, sys: 851 ms, total: 3.19 s
Wall time: 3.2 s


In [ ]:
train_ds.__len__(),valid_ds.__len__() #depends on batching

(592, 123)

In [ ]:
class CustomMLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        element_wise = tf.math.multiply_no_nan(x=y_true,
                                                   y=tf.math.divide_no_nan(x=tf.math.subtract(x=y_true, y=y_pred), y=y_pred))
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class CustomMFullLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        element_wise = tf.math.add(x=tf.math.divide_no_nan(x=tf.math.subtract(x=y_true, y=y_pred), y=y_pred),
                           y=tf.math.divide_no_nan(x=tf.math.subtract(x=1.0, y=y_true), y=tf.math.subtract(x=1.0, y=y_pred)))
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class CustomLLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        element_wise = tf.math.divide_no_nan(x=y_true,
                                             y=tf.math.sqrt(
                                                 x=tf.math.subtract(x=1.0,
                                                                    y=tf.math.squared_difference(x=y_pred, y=1.0))
                                             ))

        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class CustomLFullLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        element_wise = tf.math.subtract(x=tf.math.add(x=tf.math.divide_no_nan(x=y_true,
                                             y=tf.math.sqrt(
                                                 x=tf.math.subtract(x=1.0,
                                                                    y=tf.math.squared_difference(x=y_pred, y=1.0))
                                             )),
                                                      y=tf.math.divide_no_nan(x=tf.math.subtract(x=1.0,y=y_true),
                                                                              y=tf.math.sqrt(
                                                                                  tf.math.subtract(x=1.0,y=tf.math.square(y_pred))
                                                                              ))),
                                        y=1.0)

        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class CE_Loss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        log_y_pred = tf.math.log(y_pred)
        element_wise = -tf.math.multiply_no_nan(x=log_y_pred, y=y_true)
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class ParabolicLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        element_wise = tf.math.multiply_no_nan(x=y_true,
                                                   y=tf.math.subtract(x=1.0, y=tf.math.square(y_pred)))
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class CircleLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        element_wise = tf.math.multiply_no_nan(x=y_true,
                                                   y=tf.math.sqrt(tf.math.subtract(x=1.0, y=tf.math.square(y_pred))))
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

class CosLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call(self, y_true, y_pred):
        element_wise = tf.math.multiply_no_nan(y_true, tf.math.cos(tf.math.divide_no_nan(tf.math.multiply_no_nan(math.pi, y_pred), 2.0)))
        return tf.reduce_mean(tf.reduce_sum(element_wise,axis=1))

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import SGD, Adam

def create_model():
    base_model = VGG19(
        include_top=False,
        weights='imagenet',
        input_shape=(160, 160, 3),
        classes=10
    )

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='tanh')(x)
    predictions = Dense(10, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [ ]:
losses = [CE_Loss(),CustomMLoss(),CustomMFullLoss(),CustomLLoss(),CustomLFullLoss()]
names = ["Cross Entropy","Custom MLoss","Custom Full MLoss","Custom LLoss","Custom Full LLoss"]
for i in range(len(losses)):
    tf.keras.backend.clear_session()
    loss_fn = losses[i]
    model = create_model()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss=loss_fn, metrics=["accuracy"])
    history = model.fit(train_ds, validation_data=valid_ds, epochs=10, verbose=0)
    tr_loss, tr_acc = model.evaluate(train_ds, verbose=0)
    ts_loss, ts_acc = model.evaluate(valid_ds, verbose=0)
    print("Loss function: {0}, Train Accuracy: {1}, Test Accuracy: {2}, Train Loss: {1}, Test Loss: {2}"
    .format(names[i], tr_acc, ts_acc, tr_loss, ts_loss))

Loss function: Cross Entropy, Train Accuracy: 0.7384095191955566, Test Accuracy: 0.7785987257957458, Train Loss: 0.7384095191955566, Test Loss: 0.7785987257957458
Loss function: Custom MLoss, Train Accuracy: 0.745907723903656, Test Accuracy: 0.7836942672729492, Train Loss: 0.745907723903656, Test Loss: 0.7836942672729492
Loss function: Custom Full MLoss, Train Accuracy: 0.7478086352348328, Test Accuracy: 0.7880254983901978, Train Loss: 0.7478086352348328, Test Loss: 0.7880254983901978
Loss function: Custom LLoss, Train Accuracy: 0.7435843348503113, Test Accuracy: 0.7839490175247192, Train Loss: 0.7435843348503113, Test Loss: 0.7839490175247192
Loss function: Custom Full LLoss, Train Accuracy: 0.7550956010818481, Test Accuracy: 0.7946496605873108, Train Loss: 0.7550956010818481, Test Loss: 0.7946496605873108


In [ ]:
'''
train_loss, train_acc = model.evaluate(valid_ds, verbose=0)
test_loss, test_acc = model.evaluate(valid_ds, verbose=0)
print(f'The training accuracy was {round(train_acc,4)} and validation accuracy was {round(test_acc,4)} ')
print(f'The training loss was {round(train_loss,4)} and validation loss was {round(test_loss,4)} ')
'''

"\ntrain_loss, train_acc = model.evaluate(valid_ds, verbose=0)\ntest_loss, test_acc = model.evaluate(valid_ds, verbose=0)\nprint(f'The training accuracy was {round(train_acc,4)} and validation accuracy was {round(test_acc,4)} ')\nprint(f'The training loss was {round(train_loss,4)} and validation loss was {round(test_loss,4)} ')\n"